In [1]:
from transformers import WhisperModel, WhisperConfig
import whisper
import torch

/home/u8915687/miniconda3/envs/imagebind_LLM/lib/python3.9/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


In [2]:
model_config = WhisperConfig.from_pretrained("openai/whisper-medium")
model = WhisperModel.from_pretrained("openai/whisper-medium")

In [43]:
audio = whisper.load_audio("/work/u8915687/big-superb/big-superb-train-data/SpeechTextMatching_LibrispeechTrainClean100/train/307-127535-0008.flac")
print(len(audio))
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).unsqueeze(0)

198800


In [47]:
whisper.load_audio("/work/u8915687/big-superb/big-superb-train-data/SpeechTextMatching_LibrispeechTrainClean100/train/307-127535-0008.flac").shape

(198800,)

In [6]:
mel = mel.to("cuda")
model = model.to("cuda")

In [11]:
output = model.encoder(mel)

tensor([[[-7.9813e-01,  4.9417e-01, -1.2200e+00,  ..., -1.1205e+00,
          -3.9536e-01,  2.1145e+00],
         [-1.2252e-01,  1.3586e-01, -1.0996e+00,  ..., -2.0104e+00,
           3.0757e-02,  1.3367e+00],
         [-2.1397e+00, -1.9440e-03,  5.1249e-01,  ...,  1.2017e+00,
           9.9713e-01,  1.0283e+00],
         ...,
         [-9.3571e-02,  6.9931e-01,  1.0634e+00,  ...,  6.8936e-02,
          -8.8745e-01, -5.1617e-01],
         [-8.0370e-01,  1.1472e+00,  1.3481e+00,  ...,  1.1574e+00,
          -6.3519e-01, -6.7661e-01],
         [-1.2052e+00,  1.3521e+00,  1.4718e+00,  ...,  1.2762e+00,
          -3.9900e-01, -6.2857e-01]]], device='cuda:0',
       grad_fn=<NativeLayerNormBackward0>)

In [38]:
mel.size()

torch.Size([1, 80, 3000])

In [40]:
class trans(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.project_audio1 = torch.nn.Conv1d(1024, 1024, kernel_size=10, stride=5, padding=1)
        self.project_audio2 = torch.nn.Conv1d(1024, 1024, kernel_size=10, stride=10, padding=1)
    def forward(self, data):
        data = self.project_audio1(data)
        data = self.project_audio2(data)
        return data
trans = trans()
trans = trans.to("cuda")

print(output.last_hidden_state.size())
trans(output.last_hidden_state.transpose(1, 2)).transpose(1, 2).size()

torch.Size([1, 1500, 1024])


torch.Size([1, 30, 1024])